In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
tf.random.set_seed(42)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:  tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e: print(e)

import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.transform import resize

from sklearn.model_selection import train_test_split

from config_femnist import *
from utils.read_data_utils import *
from utils.model_utils import *
from utils.pruning_utils import *

2024-04-15 16:49:41.642777: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-15 16:49:41.642795: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-15 16:49:41.642837: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
def Define_Simple_CNN_Model(input_shape, output_shape, list_number_filters, max_pooling_step=2, model_name=None):
    """
    This function create the simple CNN model. 
    """

    kernel_size = 5
    model = models.Sequential(name=model_name)
    model.add(Input(input_shape))

    for (idx_filter, number_filter) in enumerate(list_number_filters):
        model.add(Conv2D(number_filter, (kernel_size, kernel_size), name=f'prunable_conv_{idx_filter}'))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=max_pooling_step, strides=max_pooling_step))

    model.add(layers.Flatten())
    model.add(layers.Dense(output_shape, activation='softmax', name='classifier'))
    return model

In [6]:
global_model = Define_Simple_CNN_Model(input_shape=INPUT_SHAPE, output_shape=OUPUT_SHAPE,\
                            list_number_filters=LIST_NUMBER_FILTERS, model_name="global_model")

global_model.summary()

Model: "global_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prunable_conv_0 (Conv2D)    (None, 24, 24, 32)        832       
                                                                 
 activation_4 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 12, 12, 32)        0         
 g2D)                                                            
                                                                 
 prunable_conv_1 (Conv2D)    (None, 8, 8, 64)          51264     
                                                                 
 activation_5 (Activation)   (None, 8, 8, 64)          0         
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 4, 4, 64)          0         
 g2D)                                                 

In [1]:
import cv2

In [2]:
cv2.__version__

'4.7.0'